# RXposé (Version 2.0) #

As of March, 2024, I greatly prefer working in Python over R. In this file, I will build a better pipeline than the version in R from last year, as well as (hopefully) incoroporate a better data source to be able to finish the original analysis. Ideally, this project will also produce a searchable database, allowing other contributors to perform future analysis.

## Data Collection ##

The first step is to collect better data. [This website](https://iwrp.net/) has results in PDF format going back to 1928. Since this format is not helpful for computer analysis, we need to scrape it and convert it to a .csv or similarly usable file.

In [55]:
# Imports for data collection
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

import re

import pandas as pd

import warnings

In [56]:
def download_table(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    table = soup.find_all('table')
    
    
    if table:
        df = pd.read_html(str(table))[0]
        return df
        
    else:
        print(f"No table found on page")
        return None

In [57]:
df = download_table('https://iwrp.net/')

/var/folders/84/dwt352cn4p74lsf57dxx96b00000gn/T/ipykernel_4711/291072501.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


In [58]:
print(df.head())
print(df.size)

         Date                              Name        Place Nation   Gender  \
0  2023-12-04                    IWF Grand Prix         Doha    QAT    Males   
1  2023-12-04                    IWF Grand Prix         Doha    QAT  Females   
2  2023-11-15  48 th Junior World Championships  Guadalajara    MEX    Males   
3  2023-11-15  27 th Junior World Championships  Guadalajara    MEX  Females   
4  2023-10-20  11 th Youth Polish Championships     Biłgoraj    POL    Males   

  Age category  
0       Senior  
1       Senior  
2       Junior  
3       Junior  
4     Youth 15  
15510


In [59]:
print(df[df['Gender'] == 'Males'].size)
print(df[df['Gender'] == 'Females'].size)

6276
4656


In [60]:
df.dtypes

Date            object
Name            object
Place           object
Nation          object
Gender          object
Age category    object
dtype: object

In [61]:
def is_valid_date(date_string):
    try: 
        pd.to_datetime(date_string)
        return True
    except:
        ValueError
        return False

In [20]:
# df['Date'] = pd.to_datetime(df['Date'])

# This throws and erro without fixing the one problematic input


In [62]:
valid_dates_mask = df['Date'].apply(is_valid_date)

invalid_dates = df[~valid_dates_mask]
invalid_dates

,Date,Name,Place,Nation,Gender,Age category
2538,1979-00-00,57 th European Championships,Varna,BUL,Males,Senior


In [63]:
# Quick internt search for the correct date

def replace_invalid_date(date_string):
    try: 
        pd.to_datetime(date_string)
        return date_string
    except:
        ValueError
        return '1979-05-19'
    
df['Date'] = df['Date'].apply(replace_invalid_date)

In [64]:
df['Date'] = pd.to_datetime(df['Date'])

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2585 entries, 0 to 2584
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          2585 non-null   datetime64[ns]
 1   Name          2585 non-null   object        
 2   Place         2583 non-null   object        
 3   Nation        2585 non-null   object        
 4   Gender        2585 non-null   object        
 5   Age category  2585 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 121.3+ KB


Okay, what I actually need to do for now:

1. I need to have a single dataframe with all the results, but I need the date included. I think I need to go back and get a datetime object instead of a numerical year. Then I can avoid duplicate rows, since a single lifter's total, name, and date of competition will be uniquely identifying. Not all of this data will be used in this analysis, but we want to be able to easily search for all international performances of any lifter.

2. I need to then retrieve the actual lifting information from the links. The smaller dataframe generated by importing the table(s) at each link can then be appended to the large dataframe. I NEED TO MAKE SURE THAT THE DATE AND THE WEIGHT CATEGORY ARE INCLUDED IN EACH ROW AS PART OF THIS TRANSFORMATION. This should be fairly simple. Create the data frame with the necessary columns, then populate all the scraped values as additional columns.

In [66]:
# Loop through the 'Name' column and follow the links

def download_links(url: str, events_list: list) -> pd.DataFrame:
    # Send GET request to URL
    response = requests.get(url)
    
    # Parse the HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all links on the page to sort later
    links = soup.find_all('a')
    
    return [link for link in links]
    

In [67]:
links = download_links('https://iwrp.net/', df['Name'])
print(type(links))

<class 'list'>


In [68]:
print(links[:10])
print(len(links))
print(len(df['Name']))


[<a href="/">IWRP</a>, <a href="/global-statistics">Global Statistics</a>, <a href="/../"><div class="header__logo"></div></a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2752">IWF Grand Prix</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2753">IWF Grand Prix</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2748">48 th Junior World Championships</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2749">27 th Junior World Championships</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2746">11 th Youth Polish Championships</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2747">11 th Youth Polish Championships</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2745">28 th Waldemar Malak Memorial	</a>]
2590
2585


In [69]:
links = links[3:-2]
links[:10]

[<a href="/component/cwyniki/?view=contest&amp;id_zawody=2752">IWF Grand Prix</a>,
 <a href="/component/cwyniki/?view=contest&amp;id_zawody=2753">IWF Grand Prix</a>,
 <a href="/component/cwyniki/?view=contest&amp;id_zawody=2748">48 th Junior World Championships</a>,
 <a href="/component/cwyniki/?view=contest&amp;id_zawody=2749">27 th Junior World Championships</a>,
 <a href="/component/cwyniki/?view=contest&amp;id_zawody=2746">11 th Youth Polish Championships</a>,
 <a href="/component/cwyniki/?view=contest&amp;id_zawody=2747">11 th Youth Polish Championships</a>,
 <a href="/component/cwyniki/?view=contest&amp;id_zawody=2745">28 th Waldemar Malak Memorial	</a>,
 <a href="/component/cwyniki/?view=contest&amp;id_zawody=2743">88 th World Championships</a>,
 <a href="/component/cwyniki/?view=contest&amp;id_zawody=2744">30 th World Championships</a>,
 <a href="/component/cwyniki/?view=contest&amp;id_zawody=2741">37th Asian JuniorChampionship</a>]

In [70]:
len(links)

2585

In [71]:
print(links)
print(links[::-1])

[<a href="/component/cwyniki/?view=contest&amp;id_zawody=2752">IWF Grand Prix</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2753">IWF Grand Prix</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2748">48 th Junior World Championships</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2749">27 th Junior World Championships</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2746">11 th Youth Polish Championships</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2747">11 th Youth Polish Championships</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2745">28 th Waldemar Malak Memorial	</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2743">88 th World Championships</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2744">30 th World Championships</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawody=2741">37th Asian JuniorChampionship</a>, <a href="/component/cwyniki/?view=contest&amp;id_zawod

In [72]:
df['link'] = [str(link) for link in links]
df.head()

,Date,Name,Place,Nation,Gender,Age category,link
0,2023-12-04,IWF Grand Prix,Doha,QAT,Males,Senior,"<a href=""/component/cwyniki/?view=contest&amp;..."
1,2023-12-04,IWF Grand Prix,Doha,QAT,Females,Senior,"<a href=""/component/cwyniki/?view=contest&amp;..."
2,2023-11-15,48 th Junior World Championships,Guadalajara,MEX,Males,Junior,"<a href=""/component/cwyniki/?view=contest&amp;..."
3,2023-11-15,27 th Junior World Championships,Guadalajara,MEX,Females,Junior,"<a href=""/component/cwyniki/?view=contest&amp;..."
4,2023-10-20,11 th Youth Polish Championships,Biłgoraj,POL,Males,Youth 15,"<a href=""/component/cwyniki/?view=contest&amp;..."


In [73]:
df.rename({'Name': 'Competition', 'Place': 'Host City', 'Nation': 'Host Nation', 'Age category': 'Age Category'}, axis=1, inplace=True)
df.head()

,Date,Competition,Host City,Host Nation,Gender,Age Category,link
0,2023-12-04,IWF Grand Prix,Doha,QAT,Males,Senior,"<a href=""/component/cwyniki/?view=contest&amp;..."
1,2023-12-04,IWF Grand Prix,Doha,QAT,Females,Senior,"<a href=""/component/cwyniki/?view=contest&amp;..."
2,2023-11-15,48 th Junior World Championships,Guadalajara,MEX,Males,Junior,"<a href=""/component/cwyniki/?view=contest&amp;..."
3,2023-11-15,27 th Junior World Championships,Guadalajara,MEX,Females,Junior,"<a href=""/component/cwyniki/?view=contest&amp;..."
4,2023-10-20,11 th Youth Polish Championships,Biłgoraj,POL,Males,Youth 15,"<a href=""/component/cwyniki/?view=contest&amp;..."


** Current Project **
Due to the rule changes in the sport of weighlifting, we are going to focus only on events post the Munich Olympics in 1972. This event ended on September 11, 1972, so we will filter specifically for these events. ALmost all data from this source is from after this time anyway, but it will also likely help prevent some errors when reading in the data.

In [88]:
# Some masks for faster computation

modern_event_mask = df['Date'] > '1972-09-11'
gender_mask = df['Gender'] == 'Females'
senior_mask = df['Age Category'] == 'Senior'

len(df[modern_event_mask][gender_mask][senior_mask])

/var/folders/84/dwt352cn4p74lsf57dxx96b00000gn/T/ipykernel_4711/2055256396.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(df[modern_event_mask][gender_mask][senior_mask])
/var/folders/84/dwt352cn4p74lsf57dxx96b00000gn/T/ipykernel_4711/2055256396.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(df[modern_event_mask][gender_mask][senior_mask])


359

In [118]:
# This is taking my machine about 40 seconds for 10 separate event links

def all_data(guiding_data_frame: pd.DataFrame) -> pd.DataFrame:
    
    full_dataframe = pd.DataFrame(columns= ['Date', 'Competition', 'Host City', 'Host Nation', 'Gender', 'Age Category',
                                            'Overall Rank', 'Athlete Name', 'Athlete Nation', 'Bodyweight (kg)', 'Session', 
                                            'Snatch 1', 'Snatch 2', 'Snatch 3', 'Snatch Rank',
                                            'C&J 1', 'C&J 2', 'C&J 3', 'C&J Rank',
                                            'Total (kg)', 'Sinclair'])
    
    pattern = re.compile(r'"([^"]*)"')
    for idx, link in enumerate(guiding_data_frame['link'][:5]):
        match = pattern.search(link)
        if match:
            event_link = match.group(1)
            absolute_url = 'https://iwrp.net/' + event_link

            response = requests.get(absolute_url)
            if response.status_code == 200:  # Check if request was successful
                soup = BeautifulSoup(response.content, 'html.parser')
                tables = soup.find_all('table')
                for table in tables[:1]:
                    with warnings.catch_warnings():
                        warnings.filterwarnings('ignore', category = FutureWarning)
                        try:
                            temp_df = pd.read_html(str(table))[0]
                            # Extract column names from the first row
                            temp_df.columns = temp_df.iloc[0]
                            # Drop the first row (header row)
                            temp_df = temp_df.drop(0)
                            #temp_df.columns = ['Overall Rank', 'Athlete Name', 'Athlete Nation', 'Bodyweight (kg)', 'Session', 
                                            #'Snatch 1', 'Snatch 2', 'Snatch 3', 'Snatch Rank',
                                            #'C&J 1', 'C&J 2', 'C&J 3', 'C&J Rank',
                                            #'Total (kg)', 'Sinclair']
                            
                            # Reset index to ensure uniqueness
                            temp_df.reset_index(drop=True, inplace=True)
                            
                            temp_df['Date'] = guiding_data_frame['Date'][idx]
                            temp_df['Competition'] = guiding_data_frame['Competition'][idx]
                            temp_df['Host City'] =  guiding_data_frame['Host City'][idx]
                            temp_df['Host Nation'] = guiding_data_frame['Host Nation'][idx]
                            temp_df['Gender'] = guiding_data_frame['Gender'][idx]
                            temp_df['Age Category'] = guiding_data_frame['Age Category'][idx]
                            
                            # This is to save memory for this expensive process (expensive for my laptop)
                            # temp_df = temp_df.drop_duplicates()
                            
                            # Concatenate the data frames
                            full_dataframe = pd.concat([full_dataframe, temp_df], ignore_index=True)
                            
                        except Exception as e:
                            print(f"Error processing table: {e}")
                            print(temp_df[:5])
                            continue
                        
                       
                # Implement a progress checker to watch for failing internet connection 
                if idx % 10 == 9:
                    print(f"successfully added information from page {idx+1} of {len(guiding_data_frame)}")
                        
                
            else:
                print(f"Failed to fetch data from {absolute_url}. Status code: {response.status_code}")
        else:
            print(f"No match found for link: {link}")
    
    return full_dataframe

In [120]:
#As a test, this is only Senior Women's events from after Munich 1972 (no women's events before then, mask is unnecessary)

senior_women_guiding_df = df[modern_event_mask][gender_mask][senior_mask].reset_index(drop=True)

senior_women_data = all_data(senior_women_guiding_df)

/var/folders/84/dwt352cn4p74lsf57dxx96b00000gn/T/ipykernel_4711/3043059090.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  senior_women_guiding_df = df[modern_event_mask][gender_mask][senior_mask].reset_index(drop=True)
/var/folders/84/dwt352cn4p74lsf57dxx96b00000gn/T/ipykernel_4711/3043059090.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  senior_women_guiding_df = df[modern_event_mask][gender_mask][senior_mask].reset_index(drop=True)


Error processing table: Reindexing only valid with uniquely valued Index objects
0     Pl  Surname and name  Nation    B.W    Gr.  Snatch  Snatch  Snatch  \
0     Pl  Surname and name  Nation    B.W    Gr.       1       2       3   
1      1      Won Hyon Sim     PRK  45.00      A    77.0    82.0    86.0   
2      2   Jean Ramos Rose     PHI  45.00      A    68.0    70.0    70.0   
3  49 kg             49 kg   49 kg  49 kg  49 kg   49 kg   49 kg   49 kg   
4     Pl  Surname and name  Nation    B.W    Gr.  Snatch  Snatch  Snatch   

0    NaN  Cl&Jerk  ...  Cl&Jerk    NaN    NaN  Sincler       Date  \
0    NaN        1  ...        3    NaN    NaN  Sincler 2023-12-04   
1      1     90.0  ...     99.0      1  181.0    303.2 2023-12-04   
2      2     85.0  ...     87.0      2  155.0    259.7 2023-12-04   
3  49 kg    49 kg  ...    49 kg  49 kg  49 kg    49 kg 2023-12-04   
4    NaN  Cl&Jerk  ...  Cl&Jerk    NaN    NaN  Sincler 2023-12-04   

0     Competition Host City Host Nation   Gende

Maybe I need to work with these in series individually, then create the dataframe from that. That would solve the problem of misaligned columns, and it would likely be faster.

In [105]:
senior_women_guiding_df.head()

,Date,Competition,Host City,Host Nation,Gender,Age Category,link
0,2023-12-04,IWF Grand Prix,Doha,QAT,Females,Senior,"<a href=""/component/cwyniki/?view=contest&amp;..."
1,2023-09-04,30 th World Championships,Riyadh,KSA,Females,Senior,"<a href=""/component/cwyniki/?view=contest&amp;..."
2,2023-06-15,30 th Polish Championships,Gdańsk,POL,Females,Senior,"<a href=""/component/cwyniki/?view=contest&amp;..."
3,2023-05-05,31 st Asian Championships,Jinju,KOR,Females,Senior,"<a href=""/component/cwyniki/?view=contest&amp;..."
4,2023-04-15,35 th European Championships,Yerevan,ARM,Females,Senior,"<a href=""/component/cwyniki/?view=contest&amp;..."


In [117]:
senior_women_data.head()

,Date,Competition,Host City,Host Nation,Gender,Age Category,Overall Rank,Athlete Name,Athlete Nation,Bodyweight (kg),...,Snatch 1,Snatch 2,Snatch 3,Snatch Rank,C&J 1,C&J 2,C&J 3,C&J Rank,Total (kg),Sinclair


It's working!!

In [39]:
# These are leftovers from the import process

senior_women_data[senior_women_data['Sinclair'] == 'Sincler']

,Date,Competition,Host City,Host Nation,Gender,Age Category,Overall Rank,Athlete Name,Athlete Nation,Bodyweight (kg),...,Snatch 1,Snatch 2,Snatch 3,Snatch Rank,C&J 1,C&J 2,C&J 3,C&J Rank,Total (kg),Sinclair
0,2023-12-04,IWF Grand Prix,Doha,QAT,Females,Senior,Pl,Surname and name,Nation,B.W,...,Snatch,Snatch,Snatch,NaN,Cl&Jerk,Cl&Jerk,Cl&Jerk,NaN,NaN,Sincler
1,2023-12-04,IWF Grand Prix,Doha,QAT,Females,Senior,Pl,Surname and name,Nation,B.W,...,1,2,3,NaN,1,2,3,NaN,NaN,Sincler
9,2023-12-04,IWF Grand Prix,Doha,QAT,Females,Senior,Pl,Surname and name,Nation,B.W,...,Snatch,Snatch,Snatch,NaN,Cl&Jerk,Cl&Jerk,Cl&Jerk,NaN,NaN,Sincler
10,2023-12-04,IWF Grand Prix,Doha,QAT,Females,Senior,Pl,Surname and name,Nation,B.W,...,1,2,3,NaN,1,2,3,NaN,NaN,Sincler
35,2023-12-04,IWF Grand Prix,Doha,QAT,Females,Senior,Pl,Surname and name,Nation,B.W,...,Snatch,Snatch,Snatch,NaN,Cl&Jerk,Cl&Jerk,Cl&Jerk,NaN,NaN,Sincler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694585,1987-05-31,1 st World Championships,Daytona Beach,USA,Females,Senior,Pl,Surname and name,Nation,B.W,...,1,2,3,NaN,1,2,3,NaN,NaN,Sincler
694598,1987-05-31,1 st World Championships,Daytona Beach,USA,Females,Senior,Pl,Surname and name,Nation,B.W,...,Snatch,Snatch,Snatch,NaN,Cl&Jerk,Cl&Jerk,Cl&Jerk,NaN,NaN,Sincler
694599,1987-05-31,1 st World Championships,Daytona Beach,USA,Females,Senior,Pl,Surname and name,Nation,B.W,...,1,2,3,NaN,1,2,3,NaN,NaN,Sincler
694624,1987-05-31,1 st World Championships,Daytona Beach,USA,Females,Senior,Pl,Surname and name,Nation,B.W,...,Snatch,Snatch,Snatch,NaN,Cl&Jerk,Cl&Jerk,Cl&Jerk,NaN,NaN,Sincler


In [40]:
senior_women_data = senior_women_data[senior_women_data['Sinclair'] != 'Sincler']
senior_women_data.shape

(634150, 21)

In [41]:
senior_women_data.head()

,Date,Competition,Host City,Host Nation,Gender,Age Category,Overall Rank,Athlete Name,Athlete Nation,Bodyweight (kg),...,Snatch 1,Snatch 2,Snatch 3,Snatch Rank,C&J 1,C&J 2,C&J 3,C&J Rank,Total (kg),Sinclair
2,2023-12-04,IWF Grand Prix,Doha,QAT,Females,Senior,1,Pang Un Chol,PRK,54.94,...,110.0,114.0,116.0,1,142.0,148.0,152.0,1,268.0,441.0
3,2023-12-04,IWF Grand Prix,Doha,QAT,Females,Senior,2,Nugroho Satrio Adi,INA,54.79,...,108.0,112.0,115.0,2,135.0,139.0,144.0,2,254.0,418.9
4,2023-12-04,IWF Grand Prix,Doha,QAT,Females,Senior,3,Yodage Dilanka Isuru Kumara,SRI,55.00,...,106.0,112.0,114.0,4,133.0,139.0,140.0,3,245.0,402.8
5,2023-12-04,IWF Grand Prix,Doha,QAT,Females,Senior,4,Rizqih Muhammad Ibnu,INA,55.00,...,111.0,111.0,113.0,3,130.0,134.0,134.0,4,243.0,399.5
6,2023-12-04,IWF Grand Prix,Doha,QAT,Females,Senior,5,Taj Md Ashikur Rahman,BAN,55.00,...,93.0,97.0,100.0,6,112.0,118.0,118.0,5,212.0,348.5


In [46]:
senior_women_data = senior_women_data.drop_duplicates()
senior_women_data.shape

(117700, 21)

In [51]:
len(senior_women_data['Athlete Name'].unique())

198

In [52]:
test_df = pd.read_csv('/Users/aaronkeeney/Documents/Data_Analytics_Projects/Rxpose/senior_women_data.csv')
test_df['Athlete Name'].value_counts()

Athlete Name
Murakami Eishiro                     980
Nagy Peter                           980
Varazdat Lalayan                     980
Martirosyan Simon                    980
Ziaziulin Eduard                     980
                                    ... 
Andriatsitohaina Tojonirina Alain    490
Kingue Matam Bernardin               490
81 kg                                490
Ri Chongsong                         490
Bruce Kyle John Ryan Christopher     490
Name: count, Length: 198, dtype: int64

At this point, these data could be turned into a database. I would prefer to be able to do that with all the data at once, however. I am going to save this dataframe as a .csv, then I will be able to load everything at once. For now, we will focus on only Senior men and women.

**Future Steps**
1. Turn all numeric columns from strings to numbers. Will need to look for outliers/obviously wrong values.

2. Concat dataframes and create database -- Postgres? Mysql? SQLite?

In [34]:
# Save the current data
# We can change the variable names for future use

path = '/Users/aaronkeeney/Documents/Data_Analytics_Projects/Rxpose/senior_women_data.csv'

senior_women_data.to_csv(path)



Next, we will repeat this process with senior men

In [37]:
modern_event_mask = df['Date'] > '1972-09-11'
gender_mask = df['Gender'] == 'Males'
senior_mask = df['Age Category'] == 'Senior'

senior_men_guiding_df = df[modern_event_mask][gender_mask][senior_mask].reset_index()
senior_men_guiding_df.shape

/var/folders/84/dwt352cn4p74lsf57dxx96b00000gn/T/ipykernel_2025/4076420237.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  senior_men_guiding_df = df[modern_event_mask][gender_mask][senior_mask].reset_index()
/var/folders/84/dwt352cn4p74lsf57dxx96b00000gn/T/ipykernel_2025/4076420237.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  senior_men_guiding_df = df[modern_event_mask][gender_mask][senior_mask].reset_index()


(520, 8)

In [39]:
senior_men_data = all_data(senior_men_guiding_df)

successfully added information from page 10 of 520
successfully added information from page 20 of 520
successfully added information from page 30 of 520
successfully added information from page 40 of 520
successfully added information from page 50 of 520
successfully added information from page 60 of 520
successfully added information from page 70 of 520
successfully added information from page 80 of 520
successfully added information from page 90 of 520
successfully added information from page 100 of 520
successfully added information from page 110 of 520
successfully added information from page 120 of 520
successfully added information from page 130 of 520
successfully added information from page 140 of 520
Failed to fetch data from https://iwrp.net//component/cwyniki/?view=contest&amp;id_zawody=2752. Status code: 504
successfully added information from page 150 of 520
successfully added information from page 160 of 520
successfully added information from page 170 of 520
successfully

In [40]:
senior_men_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111466 entries, 0 to 111465
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Date             111466 non-null  datetime64[ns]
 1   Competition      111466 non-null  object        
 2   Host City        111250 non-null  object        
 3   Host Nation      111466 non-null  object        
 4   Gender           111466 non-null  object        
 5   Age Category     111466 non-null  object        
 6   Overall Rank     77918 non-null   object        
 7   Athlete Name     111466 non-null  object        
 8   Athlete Nation   111466 non-null  object        
 9   Bodyweight (kg)  111466 non-null  object        
 10  Session          111466 non-null  object        
 11  Snatch 1         111466 non-null  object        
 12  Snatch 2         111466 non-null  object        
 13  Snatch 3         111466 non-null  object        
 14  Snatch Rank      892

In [41]:
senior_men_data.head()

,Date,Competition,Host City,Host Nation,Gender,Age Category,Overall Rank,Athlete Name,Athlete Nation,Bodyweight (kg),...,Snatch 1,Snatch 2,Snatch 3,Snatch Rank,C&J 1,C&J 2,C&J 3,C&J Rank,Total (kg),Sinclair
0,2023-12-04,IWF Grand Prix,Doha,QAT,Males,Senior,Pl,Surname and name,Nation,B.W,...,Snatch,Snatch,Snatch,NaN,Cl&Jerk,Cl&Jerk,Cl&Jerk,NaN,NaN,Sincler
1,2023-12-04,IWF Grand Prix,Doha,QAT,Males,Senior,Pl,Surname and name,Nation,B.W,...,1,2,3,NaN,1,2,3,NaN,NaN,Sincler
2,2023-12-04,IWF Grand Prix,Doha,QAT,Males,Senior,1,Pang Un Chol,PRK,54.94,...,110.0,114.0,116.0,1,142.0,148.0,152.0,1,268.0,441.0
3,2023-12-04,IWF Grand Prix,Doha,QAT,Males,Senior,2,Nugroho Satrio Adi,INA,54.79,...,108.0,112.0,115.0,2,135.0,139.0,144.0,2,254.0,418.9
4,2023-12-04,IWF Grand Prix,Doha,QAT,Males,Senior,3,Yodage Dilanka Isuru Kumara,SRI,55.00,...,106.0,112.0,114.0,4,133.0,139.0,140.0,3,245.0,402.8


In [42]:
senior_men_data.shape

(111466, 21)

In [44]:
senior_men_data = senior_men_data[senior_men_data['Sinclair'] != 'Sincler']
senior_men_data.shape

(110432, 21)

In [45]:
senior_men_data.head()

,Date,Competition,Host City,Host Nation,Gender,Age Category,Overall Rank,Athlete Name,Athlete Nation,Bodyweight (kg),...,Snatch 1,Snatch 2,Snatch 3,Snatch Rank,C&J 1,C&J 2,C&J 3,C&J Rank,Total (kg),Sinclair
2,2023-12-04,IWF Grand Prix,Doha,QAT,Males,Senior,1,Pang Un Chol,PRK,54.94,...,110.0,114.0,116.0,1,142.0,148.0,152.0,1,268.0,441.0
3,2023-12-04,IWF Grand Prix,Doha,QAT,Males,Senior,2,Nugroho Satrio Adi,INA,54.79,...,108.0,112.0,115.0,2,135.0,139.0,144.0,2,254.0,418.9
4,2023-12-04,IWF Grand Prix,Doha,QAT,Males,Senior,3,Yodage Dilanka Isuru Kumara,SRI,55.00,...,106.0,112.0,114.0,4,133.0,139.0,140.0,3,245.0,402.8
5,2023-12-04,IWF Grand Prix,Doha,QAT,Males,Senior,4,Rizqih Muhammad Ibnu,INA,55.00,...,111.0,111.0,113.0,3,130.0,134.0,134.0,4,243.0,399.5
6,2023-12-04,IWF Grand Prix,Doha,QAT,Males,Senior,5,Taj Md Ashikur Rahman,BAN,55.00,...,93.0,97.0,100.0,6,112.0,118.0,118.0,5,212.0,348.5


In [46]:
# Save men's data in case it gets messed up

path = '/Users/aaronkeeney/Documents/Data_Analytics_Projects/Rxpose/senior_men_data.csv'

senior_men_data.to_csv(path)

In [2]:
# In case of crashes, use this cell to reload data from source

senior_women_data = pd.read_csv('/Users/aaronkeeney/Documents/Data_Analytics_Projects/Rxpose/senior_women_data.csv')

senior_men_data = pd.read_csv( '/Users/aaronkeeney/Documents/Data_Analytics_Projects/Rxpose/senior_men_data.csv')


In [3]:
# Combining all these data into one data frame

all_senior_data = pd.concat([senior_men_data, senior_women_data], axis=0, ignore_index= True)

In [11]:
len(all_senior_data['Host City'].unique())

454

In [21]:
all_senior_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215300 entries, 0 to 215299
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Unnamed: 0       215300 non-null  int64 
 1   Date             215300 non-null  object
 2   Competition      215300 non-null  object
 3   Host City        214872 non-null  object
 4   Host Nation      215300 non-null  object
 5   Gender           215300 non-null  object
 6   Age Category     215300 non-null  object
 7   Overall Rank     149894 non-null  object
 8   Athlete Name     215300 non-null  object
 9   Athlete Nation   215300 non-null  object
 10  Bodyweight (kg)  215300 non-null  object
 11  Session          215300 non-null  object
 12  Snatch 1         215300 non-null  object
 13  Snatch 2         215300 non-null  object
 14  Snatch 3         215300 non-null  object
 15  Snatch Rank      174038 non-null  object
 16  C&J 1            215300 non-null  object
 17  C&J 2     

In [22]:
# Change data types for numeric columns

# Using errors = 'coerce' will allow us to remove all disqualifications and non-totals easily
all_senior_data[['Overall Rank','Bodyweight (kg)', 'Snatch 1', 'Snatch 2', 'Snatch 3', 'Snatch Rank', 
                 'C&J 1', 'C&J 2', 'C&J 3', 'C&J Rank', 'Total (kg)', 'Sinclair']] = \
                     all_senior_data[['Overall Rank','Bodyweight (kg)', 'Snatch 1', 'Snatch 2', 'Snatch 3', 'Snatch Rank', 
                 'C&J 1', 'C&J 2', 'C&J 3', 'C&J Rank', 'Total (kg)', 'Sinclair']].apply(pd.to_numeric, axis=1, errors = 'coerce')


In [23]:
all_senior_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215300 entries, 0 to 215299
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       215300 non-null  int64  
 1   Date             215300 non-null  object 
 2   Competition      215300 non-null  object 
 3   Host City        214872 non-null  object 
 4   Host Nation      215300 non-null  object 
 5   Gender           215300 non-null  object 
 6   Age Category     215300 non-null  object 
 7   Overall Rank     140840 non-null  float64
 8   Athlete Name     215300 non-null  object 
 9   Athlete Nation   215300 non-null  object 
 10  Bodyweight (kg)  206246 non-null  float64
 11  Session          215300 non-null  object 
 12  Snatch 1         175044 non-null  float64
 13  Snatch 2         172026 non-null  float64
 14  Snatch 3         165990 non-null  float64
 15  Snatch Rank      164984 non-null  float64
 16  C&J 1            161966 non-null  floa

In [24]:
all_senior_data.columns

Index(['Unnamed: 0', 'Date', 'Competition', 'Host City', 'Host Nation',
       'Gender', 'Age Category', 'Overall Rank', 'Athlete Name',
       'Athlete Nation', 'Bodyweight (kg)', 'Session', 'Snatch 1', 'Snatch 2',
       'Snatch 3', 'Snatch Rank', 'C&J 1', 'C&J 2', 'C&J 3', 'C&J Rank',
       'Total (kg)', 'Sinclair'],
      dtype='object')

In [25]:
# Remove phantom index column

all_senior_data = all_senior_data[['Date', 'Competition', 'Host City', 'Host Nation',
       'Gender', 'Age Category', 'Overall Rank', 'Athlete Name',
       'Athlete Nation', 'Bodyweight (kg)', 'Session', 'Snatch 1', 'Snatch 2',
       'Snatch 3', 'Snatch Rank', 'C&J 1', 'C&J 2', 'C&J 3', 'C&J Rank',
       'Total (kg)', 'Sinclair']]

In [26]:
all_senior_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215300 entries, 0 to 215299
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Date             215300 non-null  object 
 1   Competition      215300 non-null  object 
 2   Host City        214872 non-null  object 
 3   Host Nation      215300 non-null  object 
 4   Gender           215300 non-null  object 
 5   Age Category     215300 non-null  object 
 6   Overall Rank     140840 non-null  float64
 7   Athlete Name     215300 non-null  object 
 8   Athlete Nation   215300 non-null  object 
 9   Bodyweight (kg)  206246 non-null  float64
 10  Session          215300 non-null  object 
 11  Snatch 1         175044 non-null  float64
 12  Snatch 2         172026 non-null  float64
 13  Snatch 3         165990 non-null  float64
 14  Snatch Rank      164984 non-null  float64
 15  C&J 1            161966 non-null  float64
 16  C&J 2            156936 non-null  floa

At this point, we could upload the senior data to the database. Below, I am going to filter these data and preserve only the pieces necessary for the current analysis project.

In [29]:
valid_total_mask = all_senior_data['Total (kg)'] > 0

df_seniors_analysis = all_senior_data[valid_total_mask]
df_seniors_analysis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140840 entries, 0 to 215288
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Date             140840 non-null  object 
 1   Competition      140840 non-null  object 
 2   Host City        140560 non-null  object 
 3   Host Nation      140840 non-null  object 
 4   Gender           140840 non-null  object 
 5   Age Category     140840 non-null  object 
 6   Overall Rank     140840 non-null  float64
 7   Athlete Name     140840 non-null  object 
 8   Athlete Nation   140840 non-null  object 
 9   Bodyweight (kg)  140840 non-null  float64
 10  Session          140840 non-null  object 
 11  Snatch 1         140840 non-null  float64
 12  Snatch 2         139834 non-null  float64
 13  Snatch 3         134804 non-null  float64
 14  Snatch Rank      140840 non-null  float64
 15  C&J 1            140840 non-null  float64
 16  C&J 2            137822 non-null  float64
 

In [38]:
df_seniors_analysis.sort_values(by = 'Sinclair', ascending = False)

,Date,Competition,Host City,Host Nation,Gender,Age Category,Overall Rank,Athlete Name,Athlete Nation,Bodyweight (kg),...,Snatch 1,Snatch 2,Snatch 3,Snatch Rank,C&J 1,C&J 2,C&J 3,C&J Rank,Total (kg),Sinclair
73473,2012-05-10,22 nd Pan Americana Championships,Guatemala,GUA,Males,Senior,1.0,Ri Chongsong,PRK,78.00,...,160.0,160.0,165.0,1.0,200.0,206.0,209.0,1.0,374.0,484.8
210441,1997-06-24,10 th European Championships,Sevilla,ESP,Females,Senior,1.0,Ri Chongsong,PRK,78.00,...,160.0,160.0,165.0,1.0,200.0,206.0,209.0,1.0,374.0,484.8
198029,2004-10-24,1 st Under 23 Polish Championships,Siedlce,POL,Females,Under 23,1.0,Ri Chongsong,PRK,78.00,...,160.0,160.0,165.0,1.0,200.0,206.0,209.0,1.0,374.0,484.8
193321,2006-07-26,4 th European Youth Championships,Landskrona,SWE,Females,Youth 17,1.0,Ri Chongsong,PRK,78.00,...,160.0,160.0,165.0,1.0,200.0,206.0,209.0,1.0,374.0,484.8
30245,2016-05-28,Polish Team Championships 2016,Zielona Gora,POL,Males,Senior,1.0,Ri Chongsong,PRK,78.00,...,160.0,160.0,165.0,1.0,200.0,206.0,209.0,1.0,374.0,484.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10295,2018-12-19,5th International Qatar Cup,Doha,QAT,Males,Senior,17.0,Almulla Mohammed,KUW,60.32,...,80.0,NaN,NaN,20.0,100.0,NaN,NaN,19.0,180.0,275.9
149831,2015-07-11,Pan Americana Games,Torinto,CAN,Females,Senior,17.0,Almulla Mohammed,KUW,60.32,...,80.0,NaN,NaN,20.0,100.0,NaN,NaN,19.0,180.0,275.9
195199,2005-11-09,17 th World Championships,Doha,QAT,Females,Senior,17.0,Almulla Mohammed,KUW,60.32,...,80.0,NaN,NaN,20.0,100.0,NaN,NaN,19.0,180.0,275.9
125649,2017-07-21,24 th Asian JuniorChampionship,Kathmandu,NEP,Females,Junior,17.0,Almulla Mohammed,KUW,60.32,...,80.0,NaN,NaN,20.0,100.0,NaN,NaN,19.0,180.0,275.9


Something is definitely wrong here, as there should be other lifters with Sinclair values closer to 500...